<a href="https://www.kaggle.com/code/ferhat00/isa-portfolio-analyzer-kaggle-claude?scriptVersionId=273388804" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Comprehensive Stock Portfolio Analysis
## Using QuantStats, Plotly, Matplotlib & Pandas

This notebook provides in-depth analysis of your stock portfolio including:
- Overall portfolio performance metrics
- Individual stock contributions
- Risk-adjusted returns
- Drawdown analysis
- Correlation matrices
- Interactive visualizations

In [ ]:
# Install required packages (run once)
!pip install quantstats yfinance plotly pandas-datareader matplotlib seaborn scipy

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import quantstats as qs
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Enable QuantStats mode
qs.extend_pandas()

print("✅ All libraries imported successfully!")

## ⚡ Quick Start - Run This Cell First
This cell runs all the setup steps in one go. After this completes, you can run any other cell in the notebook.

In [ ]:
# === QUICK START: RUN ALL PREREQUISITES ===\n",
print("🚀 Starting comprehensive portfolio analysis setup...\n")

# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import quantstats as qs
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
qs.extend_pandas()

print("✅ Libraries imported\n")

# Define portfolio
portfolio_data = {
    'Ticker': ['NVDA', 'SGLN', 'OKLO', 'BABA', 'RR.L', 'AMD', 'AEM', 'CRWV', 
               'BY6.F', 'TSLA', 'VKTX', 'CRCL', 'SAP', 'LAC', 'PLS.AX', 'NIO', 
               'ENX', 'LEU', 'DRO.AX', 'LAR', 'GOOGL', 'JPM', 'AMZN', 'KGC', 
               'NBIS', 'ORCL', 'PAAS', 'PLTR', 'WYFI', 'AAPL', 'BARC.L', 'LTR.AX', 
               'LKY.AX', 'PMET.TO', 'IBGL'],
    'Shares': [124, 159, 115, 50, 350, 30, 30, 20, 
               170, 10, 110, 15, 7, 700, 1990, 500, 
               10, 10, 1000, 800, 40, 20, 30, 180,
               40, 10, 40, 15, 70, 10, 1000, 1400,
               2000, 750, 35],
    'Current_Price': [153.07, 47.34, 52.45, 114.27, 9.39, 136.02, 162.24, 156.42,
                      15.13, 300.90, 26.71, 190.58, 255.60, 2.66, 1.355, 3.51,
                      145.1, 169.55, 2.56, 2.10, 281.19, 311.12, 244.22, 23.24,
                      130.82, 262.61, 35.21, 200.47, 33.99, 270.37, 4.07, 1.175,
                      0.345, 3.77, 150.38]
}

portfolio_df = pd.DataFrame(portfolio_data)
portfolio_df['Position_Value'] = portfolio_df['Shares'] * portfolio_df['Current_Price']
portfolio_df['Weight'] = portfolio_df['Position_Value'] / portfolio_df['Position_Value'].sum()
portfolio_df['Weight_Pct'] = portfolio_df['Weight'] * 100
portfolio_df = portfolio_df.sort_values('Position_Value', ascending=False).reset_index(drop=True)

print(f"✅ Portfolio defined: ${portfolio_df['Position_Value'].sum():,.2f}\n")

# Download data
tickers = portfolio_df['Ticker'].tolist()
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

print(f"📥 Downloading data from {start_date} to {end_date}...\n")

price_data_dict = {}
failed_tickers = []
successful_tickers = []

for ticker in tickers:
    try:
        print(f"  {ticker}...", end=' ')
        ticker_obj = yf.Ticker(ticker)
        hist = ticker_obj.history(start=start_date, end=end_date)
        
        if hist.empty or len(hist) < 10:
            print(f"❌ No data")
            failed_tickers.append(ticker)
        else:
            if 'Close' in hist.columns:
                price_data_dict[ticker] = hist['Close']
            elif 'Adj Close' in hist.columns:
                price_data_dict[ticker] = hist['Adj Close']
            else:
                price_data_dict[ticker] = hist.iloc[:, 0]
            
            successful_tickers.append(ticker)
            print(f"✅ {len(hist)} days")
    except Exception as e:
        print(f"❌ Error")
        failed_tickers.append(ticker)

if price_data_dict:
    price_data = pd.DataFrame(price_data_dict)
    price_data = price_data.ffill().bfill()
    
    if failed_tickers:
        print(f"\n⚠️  Excluding {len(failed_tickers)} failed tickers: {', '.join(failed_tickers)}")
        portfolio_df = portfolio_df[~portfolio_df['Ticker'].isin(failed_tickers)].reset_index(drop=True)
        portfolio_df['Position_Value'] = portfolio_df['Shares'] * portfolio_df['Current_Price']
        portfolio_df['Weight'] = portfolio_df['Position_Value'] / portfolio_df['Position_Value'].sum()
        portfolio_df['Weight_Pct'] = portfolio_df['Weight'] * 100
        tickers = portfolio_df['Ticker'].tolist()
    
    print(f"\n✅ Successfully loaded {len(successful_tickers)} stocks")
    print(f"   Date range: {price_data.index[0].date()} to {price_data.index[-1].date()}")
    
    # Calculate returns
    returns = price_data.pct_change().dropna()
    weights = portfolio_df.set_index('Ticker')['Weight'].reindex(returns.columns).fillna(0)
    portfolio_returns = (returns * weights).sum(axis=1)
    cumulative_returns = (1 + returns).cumprod()
    portfolio_cumulative_returns = (1 + portfolio_returns).cumprod()
    initial_portfolio_value = portfolio_df['Position_Value'].sum()
    portfolio_value_over_time = portfolio_cumulative_returns * initial_portfolio_value
    
    # Download benchmark
    benchmark = yf.download('^GSPC', start=start_date, end=end_date, progress=False)['Close']
    benchmark_returns = benchmark.pct_change().dropna()
    
    print(f"\n✅ Returns calculated")
    print(f"   Total Return: {(portfolio_cumulative_returns.iloc[-1] - 1) * 100:.2f}%")
    print(f"   Portfolio Value: ${portfolio_value_over_time.iloc[-1]:,.2f}")
    print(f"\n" + "="*70)
    print("🎉 SETUP COMPLETE! You can now run any cell in the notebook.")
    print("="*70)
else:
    raise ValueError("❌ Could not download data for any tickers")

---
# Detailed Analysis Sections
After running the Quick Start cell above, you can run any of the sections below.

## 1. Portfolio Definition
Define your current portfolio holdings based on the uploaded images

In [ ]:
# Portfolio Holdings - Current Snapshot (Image 1)
portfolio_data = portfolio_data

# Additional positions from Image 2 (if different timeframe)
portfolio_data_recent = {
    'Ticker': ['NVDA', 'SGLN', 'GOOGL', 'AMZN', 'IBGL', 'JPM', 'AMD', 'RR.L', 
               'BARC.L', 'NBIS', 'PLS.AX', 'KGC', 'OKLO', 'LEU', 'PLTR', 
               'AAPL', 'ORCL', 'LAR', 'DRO.AX', 'WYFI', 'LAC'],
    'Recent_Price': [205.22, 59.17, 283.26, 245.60, 15.37, 312.10, 258.66, 1170.00,
                     407.30, 130.91, 3.13, 23.24, 131.18, 359.44, 203.90,
                     271.12, 263.66, 4.34, 3.83, 33.25, 5.40]
}

portfolio_data_recent = {
    'Ticker': ['NVDA', 'SGLN', 'OKLO', 'BABA', 'RR.L', 'AMD', 'AEM', 'CRWV', 
               'BY6.F', 'TSLA', 'VKTX', 'CRCL', 'SAP', 'LAC', 'PLS.AX', 'NIO', 
               'ENX', 'LEU', 'DRO.AX', 'LAR', 'GOOGL', 'JPM', 'AMZN', 'KGC', 
               'NBIS', 'ORCL', 'PAAS', 'PLTR', 'WYFI', 'AAPL', 'BARC.L', 'LTR.AX', 
               'LKY.AX', 'PMET.TO', 'IBGL'],
    'Recent_Price': [153.07, 47.34, 52.45, 114.27, 9.39, 136.02, 162.24, 156.42,
                      15.13, 300.90, 26.71, 190.58, 255.60, 2.66, 1.355, 3.51,
                      145.1, 169.55, 2.56, 2.10, 281.19, 311.12, 244.22, 23.24,
                      130.82, 262.61, 35.21, 200.47, 33.99, 270.37, 4.07, 1.175,
                      0.345, 3.77, 150.38]
}

# Create portfolio DataFrame
portfolio_df = pd.DataFrame(portfolio_data)

# Calculate position values
portfolio_df['Position_Value'] = portfolio_df['Shares'] * portfolio_df['Current_Price']
portfolio_df['Weight'] = portfolio_df['Position_Value'] / portfolio_df['Position_Value'].sum()
portfolio_df['Weight_Pct'] = portfolio_df['Weight'] * 100

# Sort by position value
portfolio_df = portfolio_df.sort_values('Position_Value', ascending=False).reset_index(drop=True)

print(f"\n📊 Portfolio Summary")
print(f"{'='*60}")
print(f"Total Portfolio Value: ${portfolio_df['Position_Value'].sum():,.2f}")
print(f"Number of Holdings: {len(portfolio_df)}")
print(f"\nTop 5 Holdings:")
print(portfolio_df[['Ticker', 'Shares', 'Position_Value', 'Weight_Pct']].head())

In [ ]:
# Display full portfolio
portfolio_df.style.format({
    'Current_Price': '${:.2f}',
    'Position_Value': '${:,.2f}',
    'Weight': '{:.4f}',
    'Weight_Pct': '{:.2f}%'
}).background_gradient(subset=['Weight_Pct'], cmap='RdYlGn')

In [ ]:
# Optional: Verify ticker symbols before downloading (run this if you have issues)
print("🔍 Verifying ticker symbols...\n")

for ticker in portfolio_df['Ticker'].tolist():
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        if 'symbol' in info or 'longName' in info:
            name = info.get('longName', info.get('shortName', 'Unknown'))
            print(f"✅ {ticker:12} - {name[:50]}")
        else:
            print(f"⚠️  {ticker:12} - May not be valid")
    except:
        print(f"❌ {ticker:12} - Invalid ticker")

print("\n💡 Tip: If tickers show as invalid, you may need to adjust the symbols:")
print("   - UK stocks: Add .L suffix (e.g., RR.L for Rolls-Royce)")
print("   - Australian stocks: Add .AX suffix (e.g., PLS.AX)")
print("   - Some tickers may use ADRs instead (e.g., BABA for Alibaba)")

## 2. Download Historical Price Data

In [ ]:
# Download historical data for all tickers
tickers = portfolio_df['Ticker'].tolist()

# Define time period (adjust as needed)
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

print(f"📥 Downloading historical data from {start_date} to {end_date}...")
print(f"Tickers: {', '.join(tickers)}\n")

# Download tickers ONE BY ONE to handle errors gracefully
price_data_dict = {}
failed_tickers = []
successful_tickers = []

for ticker in tickers:
    try:
        print(f"Downloading {ticker}...", end=' ')
        ticker_obj = yf.Ticker(ticker)
        hist = ticker_obj.history(start=start_date, end=end_date)
        
        if hist.empty or len(hist) < 10:
            print(f"❌ No data")
            failed_tickers.append(ticker)
        else:
            # Use Close price, fallback to Adj Close if available
            if 'Close' in hist.columns:
                price_data_dict[ticker] = hist['Close']
            elif 'Adj Close' in hist.columns:
                price_data_dict[ticker] = hist['Adj Close']
            else:
                price_data_dict[ticker] = hist.iloc[:, 0]  # Use first column
            
            successful_tickers.append(ticker)
            print(f"✅ ({len(hist)} days)")
    except Exception as e:
        print(f"❌ Error: {str(e)[:50]}")
        failed_tickers.append(ticker)

# Combine all successful downloads into a single DataFrame
if price_data_dict:
    price_data = pd.DataFrame(price_data_dict)
    
    # Align all dates and forward fill missing data
    price_data = price_data.ffill().bfill()
    
    print(f"\n{'='*70}")
    print(f"✅ Successfully downloaded {len(successful_tickers)} tickers")
    print(f"Successful: {', '.join(successful_tickers)}")
    
    if failed_tickers:
        print(f"\n⚠️  Failed to download {len(failed_tickers)} tickers")
        print(f"Failed: {', '.join(failed_tickers)}")
        print(f"\nThese tickers will be excluded from analysis.")
        
        # Update portfolio_df to remove failed tickers
        portfolio_df = portfolio_df[~portfolio_df['Ticker'].isin(failed_tickers)].reset_index(drop=True)
        # Recalculate weights
        portfolio_df['Position_Value'] = portfolio_df['Shares'] * portfolio_df['Current_Price']
        portfolio_df['Weight'] = portfolio_df['Position_Value'] / portfolio_df['Position_Value'].sum()
        portfolio_df['Weight_Pct'] = portfolio_df['Weight'] * 100
        tickers = portfolio_df['Ticker'].tolist()
    
    print(f"\n📊 Data Summary:")
    print(f"Date range: {price_data.index[0].date()} to {price_data.index[-1].date()}")
    print(f"Total trading days: {len(price_data)}")
    print(f"{'='*70}\n")
else:
    raise ValueError("❌ Could not download data for any tickers. Please check ticker symbols.")

## 3. Calculate Portfolio Returns

In [ ]:
# Safety check: Ensure data was downloaded
if 'price_data' not in locals() or price_data is None or price_data.empty:
    raise ValueError("\n❌ ERROR: Price data not available. Please run the data download cell above first.\n"
                     "Make sure at least some tickers downloaded successfully.")

print(f"✅ Found price data for {len(price_data.columns)} stocks\n")

# Calculate individual stock returns
returns = price_data.pct_change().dropna()

# Calculate weighted portfolio returns
weights = portfolio_df.set_index('Ticker')['Weight'].reindex(returns.columns).fillna(0)
portfolio_returns = (returns * weights).sum(axis=1)

# Calculate cumulative returns
cumulative_returns = (1 + returns).cumprod()
portfolio_cumulative_returns = (1 + portfolio_returns).cumprod()

# Calculate portfolio value over time
initial_portfolio_value = portfolio_df['Position_Value'].sum()
portfolio_value_over_time = portfolio_cumulative_returns * initial_portfolio_value

print(f"📈 Returns calculated successfully!")
print(f"\nPortfolio Performance Summary:")
print(f"{'='*60}")
print(f"Total Return: {(portfolio_cumulative_returns.iloc[-1] - 1) * 100:.2f}%")
print(f"Current Portfolio Value: ${portfolio_value_over_time.iloc[-1]:,.2f}")
print(f"Initial Portfolio Value: ${initial_portfolio_value:,.2f}")
print(f"Absolute Gain/Loss: ${(portfolio_value_over_time.iloc[-1] - initial_portfolio_value):,.2f}")

## 4. Portfolio Allocation Visualization

In [ ]:
# Interactive Pie Chart with Plotly
fig = px.pie(portfolio_df, 
             values='Position_Value', 
             names='Ticker',
             title='Portfolio Allocation by Position Value',
             hover_data=['Shares'],
             color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=600, showlegend=True)
fig.show()

# Treemap for hierarchical view
fig2 = px.treemap(portfolio_df,
                  path=['Ticker'],
                  values='Position_Value',
                  title='Portfolio Treemap View',
                  color='Weight_Pct',
                  color_continuous_scale='RdYlGn')
fig2.update_layout(height=500)
fig2.show()

In [ ]:
# Bar chart of position values
fig = px.bar(portfolio_df, 
             x='Ticker', 
             y='Position_Value',
             title='Position Values by Stock',
             color='Weight_Pct',
             color_continuous_scale='Viridis',
             text='Position_Value')

fig.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
fig.update_layout(height=500, xaxis_tickangle=-45)
fig.show()

## 5. Portfolio Performance Over Time

In [ ]:
# Interactive portfolio value chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=portfolio_value_over_time.index,
    y=portfolio_value_over_time.values,
    mode='lines',
    name='Portfolio Value',
    line=dict(color='#00CC96', width=3),
    fill='tonexty',
    fillcolor='rgba(0, 204, 150, 0.1)'
))

fig.add_hline(y=initial_portfolio_value, 
              line_dash="dash", 
              line_color="red",
              annotation_text=f"Initial Value: ${initial_portfolio_value:,.0f}")

fig.update_layout(
    title='Portfolio Value Over Time',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    height=500,
    hovermode='x unified',
    template='plotly_dark'
)

fig.show()

In [ ]:
# Cumulative returns comparison
fig = go.Figure()

# Add portfolio
fig.add_trace(go.Scatter(
    x=portfolio_cumulative_returns.index,
    y=(portfolio_cumulative_returns - 1) * 100,
    mode='lines',
    name='Portfolio',
    line=dict(color='gold', width=4)
))

# Add top 5 holdings
top_5_tickers = portfolio_df.head(5)['Ticker'].tolist()
for ticker in top_5_tickers:
    if ticker in cumulative_returns.columns:
        fig.add_trace(go.Scatter(
            x=cumulative_returns.index,
            y=(cumulative_returns[ticker] - 1) * 100,
            mode='lines',
            name=ticker,
            opacity=0.7
        ))

fig.update_layout(
    title='Cumulative Returns: Portfolio vs Top Holdings',
    xaxis_title='Date',
    yaxis_title='Cumulative Return (%)',
    height=600,
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

## 6. QuantStats Performance Analysis

In [ ]:
# Download benchmark data (S&P 500)
benchmark = yf.download('^GSPC', start=start_date, end=end_date, progress=False)
benchmark_returns = benchmark.pct_change().dropna()

print("✅ Benchmark data downloaded (S&P 500)")

In [ ]:
benchmark.head()

In [ ]:
# Generate comprehensive QuantStats report
print("📊 Generating QuantStats Performance Metrics...\n")

# Key metrics
print("=" * 70)
print("PORTFOLIO PERFORMANCE METRICS")
print("=" * 70)

metrics = {
    'Total Return': qs.stats.comp(portfolio_returns),
    'CAGR': qs.stats.cagr(portfolio_returns),
    'Sharpe Ratio': qs.stats.sharpe(portfolio_returns),
    'Sortino Ratio': qs.stats.sortino(portfolio_returns),
    'Max Drawdown': qs.stats.max_drawdown(portfolio_returns),
    'Calmar Ratio': qs.stats.calmar(portfolio_returns),
    'Volatility (Annual)': qs.stats.volatility(portfolio_returns),
    'Value at Risk (95%)': qs.stats.var(portfolio_returns),
    'Conditional VaR (95%)': qs.stats.cvar(portfolio_returns),
    'Win Rate': qs.stats.win_rate(portfolio_returns),
    'Best Day': qs.stats.best(portfolio_returns),
    'Worst Day': qs.stats.worst(portfolio_returns),
    'Average Return': portfolio_returns.mean(),
    'Average Win': portfolio_returns[portfolio_returns > 0].mean(),
    'Average Loss': portfolio_returns[portfolio_returns < 0].mean(),
}

for metric, value in metrics.items():
    if 'Return' in metric or 'CAGR' in metric or 'Drawdown' in metric or 'Day' in metric:
        print(f"{metric:.<50} {value*100:>8.2f}%")
    else:
        print(f"{metric:.<50} {value:>8.4f}")

print("=" * 70)

In [ ]:
# QuantStats visualization suite
qs.plots.snapshot(portfolio_returns, title='Portfolio Performance Snapshot')
plt.show()

In [ ]:
# Returns distribution
qs.plots.histogram(portfolio_returns)
plt.show()

In [ ]:
# Drawdown analysis
qs.plots.drawdown(portfolio_returns)
plt.show()

qs.plots.drawdowns_periods(portfolio_returns)
plt.show()

In [ ]:
# Rolling metrics - Generate separately since QuantStats doesn't support ax parameter
print("📊 Generating Rolling Performance Metrics...\n")

# Rolling Sharpe
plt.figure(figsize=(12, 5))
qs.plots.rolling_sharpe(portfolio_returns)


# Rolling Volatility
plt.figure(figsize=(12, 5))
qs.plots.rolling_volatility(portfolio_returns)


# Rolling Beta
plt.figure(figsize=(12, 5))
qs.plots.rolling_beta(portfolio_returns, benchmark_returns)


# Rolling Sortino
plt.figure(figsize=(12, 5))
qs.plots.rolling_sortino(portfolio_returns)


print("✅ Rolling metrics generated")

## 7. Individual Stock Performance Analysis

In [ ]:
# Calculate metrics for each stock
stock_metrics = pd.DataFrame(index=tickers)

for ticker in tickers:
    if ticker in returns.columns:
        stock_returns = returns[ticker].dropna()
        
        stock_metrics.loc[ticker, 'Total Return (%)'] = (cumulative_returns[ticker].iloc[-1] - 1) * 100
        stock_metrics.loc[ticker, 'CAGR (%)'] = qs.stats.cagr(stock_returns) * 100
        stock_metrics.loc[ticker, 'Volatility (%)'] = qs.stats.volatility(stock_returns) * 100
        stock_metrics.loc[ticker, 'Sharpe Ratio'] = qs.stats.sharpe(stock_returns)
        stock_metrics.loc[ticker, 'Sortino Ratio'] = qs.stats.sortino(stock_returns)
        stock_metrics.loc[ticker, 'Max Drawdown (%)'] = qs.stats.max_drawdown(stock_returns) * 100
        stock_metrics.loc[ticker, 'Win Rate (%)'] = qs.stats.win_rate(stock_returns) * 100
        stock_metrics.loc[ticker, 'Best Day (%)'] = qs.stats.best(stock_returns) * 100
        stock_metrics.loc[ticker, 'Worst Day (%)'] = qs.stats.worst(stock_returns) * 100

# Merge with portfolio weights
stock_metrics = stock_metrics.merge(portfolio_df[['Ticker', 'Weight_Pct', 'Position_Value']], 
                                    left_index=True, right_on='Ticker')
stock_metrics = stock_metrics.set_index('Ticker')

# Sort by position value
stock_metrics = stock_metrics.sort_values('Position_Value', ascending=False)

print("\n📊 Individual Stock Performance Metrics\n")
stock_metrics.style.format({
    'Total Return (%)': '{:.2f}',
    'CAGR (%)': '{:.2f}',
    'Volatility (%)': '{:.2f}',
    'Sharpe Ratio': '{:.2f}',
    'Sortino Ratio': '{:.2f}',
    'Max Drawdown (%)': '{:.2f}',
    'Win Rate (%)': '{:.2f}',
    'Position_Value': '${:,.2f}',
    'Weight_Pct': '{:.2f}%'
}).background_gradient(subset=['Sharpe Ratio'], cmap='RdYlGn')

In [ ]:
# Display full metrics table
display(stock_metrics)

In [ ]:
# Visualize Sharpe Ratios
fig = px.bar(stock_metrics.reset_index(), 
             x='Ticker', 
             y='Sharpe Ratio',
             title='Sharpe Ratio by Stock (Risk-Adjusted Returns)',
             color='Sharpe Ratio',
             color_continuous_scale='RdYlGn',
             text='Sharpe Ratio')

fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(height=500, xaxis_tickangle=-45)
fig.show()

In [ ]:
# Risk-Return scatter plot
fig = px.scatter(stock_metrics.reset_index(),
                 x='Volatility (%)',
                 y='CAGR (%)',
                 size='Position_Value',
                 color='Sharpe Ratio',
                 hover_name='Ticker',
                 title='Risk-Return Profile (Bubble Size = Position Value)',
                 labels={'Volatility (%)': 'Risk (Volatility %)', 'CAGR (%)': 'Return (CAGR %)'},
                 color_continuous_scale='RdYlGn',
                 size_max=60)

fig.update_layout(height=600)
fig.show()

## 8. Portfolio Contribution Analysis

In [ ]:
# Calculate contribution to portfolio returns
weighted_returns = returns * weights

# Cumulative contribution over time
cumulative_contributions = (1 + weighted_returns).cumprod()

# Total contribution
total_contributions = pd.DataFrame({
    'Ticker': tickers,
    'Weight (%)': [weights[t] * 100 for t in tickers],
    'Total Contribution (%)': [(cumulative_contributions[t].iloc[-1] - 1) * 100 if t in cumulative_contributions.columns else 0 for t in tickers],
    'Contribution to Portfolio': [weighted_returns[t].sum() * 100 if t in weighted_returns.columns else 0 for t in tickers]
}).sort_values('Contribution to Portfolio', ascending=False)

print("\n💰 Stock Contribution to Overall Portfolio Performance\n")
display(total_contributions)

In [ ]:
# Waterfall chart of contributions
fig = go.Figure(go.Waterfall(
    name="Contribution",
    orientation="v",
    x=total_contributions['Ticker'],
    y=total_contributions['Contribution to Portfolio'],
    text=[f"{v:.2f}%" for v in total_contributions['Contribution to Portfolio']],
    textposition="outside",
    connector={"line": {"color": "rgb(63, 63, 63)"}},
))

fig.update_layout(
    title="Individual Stock Contribution to Portfolio Returns",
    height=600,
    xaxis_tickangle=-45,
    yaxis_title="Contribution (%)"
)

fig.show()

In [ ]:
# Stacked area chart of contributions over time
fig = go.Figure()

for ticker in total_contributions.head(10)['Ticker']:  # Top 10 contributors
    if ticker in weighted_returns.columns:
        cum_contribution = (1 + weighted_returns[ticker]).cumprod()
        fig.add_trace(go.Scatter(
            x=cum_contribution.index,
            y=cum_contribution,
            name=ticker,
            stackgroup='one',
            mode='lines'
        ))

fig.update_layout(
    title='Cumulative Contribution to Portfolio Value Over Time (Top 10)',
    xaxis_title='Date',
    yaxis_title='Cumulative Value',
    height=600,
    hovermode='x unified'
)

fig.show()

## 9. Correlation and Diversification Analysis

In [ ]:
# Calculate correlation matrix
correlation_matrix = returns.corr()

# Plotly heatmap
fig = px.imshow(correlation_matrix,
                labels=dict(color="Correlation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1,
                title='Stock Returns Correlation Matrix',
                aspect='auto')

fig.update_layout(height=800, width=900)
fig.show()

In [ ]:
# Matplotlib correlation heatmap with annotations
plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, 
            annot=True, 
            fmt='.2f', 
            cmap='coolwarm', 
            center=0,
            square=True,
            linewidths=0.5,
            cbar_kws={"shrink": 0.8})
plt.title('Detailed Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Diversification metrics
print("\n🎯 Portfolio Diversification Metrics\n")
print("=" * 60)

# Effective number of stocks (inverse of Herfindahl index)
herfindahl_index = (weights ** 2).sum()
effective_stocks = 1 / herfindahl_index

print(f"Number of Holdings: {len(portfolio_df)}")
print(f"Effective Number of Stocks: {effective_stocks:.2f}")
print(f"Herfindahl Index: {herfindahl_index:.4f}")
print(f"\nTop 3 Holdings Weight: {portfolio_df.head(3)['Weight_Pct'].sum():.2f}%")
print(f"Top 5 Holdings Weight: {portfolio_df.head(5)['Weight_Pct'].sum():.2f}%")
print(f"Top 10 Holdings Weight: {portfolio_df.head(10)['Weight_Pct'].sum():.2f}%")

# Average correlation
avg_correlation = correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)].mean()
print(f"\nAverage Pairwise Correlation: {avg_correlation:.4f}")

print("=" * 60)

## 10. Monthly and Yearly Returns Analysis

In [ ]:
# Monthly returns heatmap
qs.plots.monthly_heatmap(portfolio_returns)
plt.show()

In [ ]:
# Yearly returns bar chart
yearly_returns = portfolio_returns.resample('Y').apply(lambda x: (1 + x).prod() - 1)

fig = go.Figure(data=[
    go.Bar(x=yearly_returns.index.year,
           y=yearly_returns.values * 100,
           marker_color=np.where(yearly_returns.values > 0, 'green', 'red'),
           text=[f"{v*100:.2f}%" for v in yearly_returns.values],
           textposition='outside')
])

fig.update_layout(
    title='Annual Returns by Year',
    xaxis_title='Year',
    yaxis_title='Return (%)',
    height=500,
    showlegend=False
)

fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.show()

In [ ]:
# Distribution of daily returns
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=['Portfolio Daily Returns Distribution', 'Q-Q Plot'])

# Histogram
fig.add_trace(
    go.Histogram(x=portfolio_returns * 100, 
                 nbinsx=50,
                 name='Returns',
                 marker_color='lightblue'),
    row=1, col=1
)

# Q-Q plot data
from scipy import stats
theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(portfolio_returns)))
sample_quantiles = np.sort(portfolio_returns.values)

fig.add_trace(
    go.Scatter(x=theoretical_quantiles,
               y=sample_quantiles,
               mode='markers',
               name='Q-Q',
               marker=dict(color='coral')),
    row=1, col=2
)

# Add reference line for Q-Q plot
fig.add_trace(
    go.Scatter(x=theoretical_quantiles,
               y=theoretical_quantiles * portfolio_returns.std() + portfolio_returns.mean(),
               mode='lines',
               name='Normal',
               line=dict(color='red', dash='dash')),
    row=1, col=2
)

fig.update_xaxes(title_text="Daily Return (%)", row=1, col=1)
fig.update_xaxes(title_text="Theoretical Quantiles", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_yaxes(title_text="Sample Quantiles", row=1, col=2)

fig.update_layout(height=500, showlegend=True, title_text="Returns Distribution Analysis")
fig.show()

## 11. Benchmark Comparison (vs S&P 500)

In [ ]:
# Compare portfolio vs benchmark
benchmark_cumulative = (1 + benchmark_returns).cumprod()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=portfolio_cumulative_returns.index,
    y=(portfolio_cumulative_returns - 1) * 100,
    mode='lines',
    name='Your Portfolio',
    line=dict(color='#00CC96', width=3)
))

fig.add_trace(go.Scatter(
    x=benchmark_cumulative.index,
    y=(benchmark_cumulative - 1) * 100,
    mode='lines',
    name='S&P 500 (SPY)',
    line=dict(color='#636EFA', width=3, dash='dash')
))

fig.update_layout(
    title='Portfolio Performance vs S&P 500 Benchmark',
    xaxis_title='Date',
    yaxis_title='Cumulative Return (%)',
    height=600,
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

In [ ]:
# Helper function to extract scalar values
def get_scalar(value):
    """Extract scalar from Series or return value as-is"""
    if isinstance(value, pd.Series):
        return value.iloc[0] if len(value) > 0 else value.values[0]
    return value

# Calculate all metrics and extract scalars
portfolio_metrics = {
    'comp': get_scalar(qs.stats.comp(portfolio_returns)) * 100,
    'cagr': get_scalar(qs.stats.cagr(portfolio_returns)) * 100,
    'volatility': get_scalar(qs.stats.volatility(portfolio_returns)) * 100,
    'sharpe': get_scalar(qs.stats.sharpe(portfolio_returns)),
    'sortino': get_scalar(qs.stats.sortino(portfolio_returns)),
    'max_dd': get_scalar(qs.stats.max_drawdown(portfolio_returns)) * 100,
    'win_rate': get_scalar(qs.stats.win_rate(portfolio_returns)) * 100,
    'best': get_scalar(qs.stats.best(portfolio_returns)) * 100,
    'worst': get_scalar(qs.stats.worst(portfolio_returns)) * 100
}

benchmark_metrics = {
    'comp': get_scalar(qs.stats.comp(benchmark_returns)) * 100,
    'cagr': get_scalar(qs.stats.cagr(benchmark_returns)) * 100,
    'volatility': get_scalar(qs.stats.volatility(benchmark_returns)) * 100,
    'sharpe': get_scalar(qs.stats.sharpe(benchmark_returns)),
    'sortino': get_scalar(qs.stats.sortino(benchmark_returns)),
    'max_dd': get_scalar(qs.stats.max_drawdown(benchmark_returns)) * 100,
    'win_rate': get_scalar(qs.stats.win_rate(benchmark_returns)) * 100,
    'best': get_scalar(qs.stats.best(benchmark_returns)) * 100,
    'worst': get_scalar(qs.stats.worst(benchmark_returns)) * 100
}

# Comparative metrics table
comparison = pd.DataFrame({
    'Metric': ['Total Return', 'CAGR', 'Volatility', 'Sharpe Ratio', 'Sortino Ratio', 
               'Max Drawdown', 'Win Rate', 'Best Day', 'Worst Day'],
    'Your Portfolio': [
        f"{portfolio_metrics['comp']:.2f}%",
        f"{portfolio_metrics['cagr']:.2f}%",
        f"{portfolio_metrics['volatility']:.2f}%",
        f"{portfolio_metrics['sharpe']:.2f}",
        f"{portfolio_metrics['sortino']:.2f}",
        f"{portfolio_metrics['max_dd']:.2f}%",
        f"{portfolio_metrics['win_rate']:.2f}%",
        f"{portfolio_metrics['best']:.2f}%",
        f"{portfolio_metrics['worst']:.2f}%"
    ],
    'S&P 500': [
        f"{benchmark_metrics['comp']:.2f}%",
        f"{benchmark_metrics['cagr']:.2f}%",
        f"{benchmark_metrics['volatility']:.2f}%",
        f"{benchmark_metrics['sharpe']:.2f}",
        f"{benchmark_metrics['sortino']:.2f}",
        f"{benchmark_metrics['max_dd']:.2f}%",
        f"{benchmark_metrics['win_rate']:.2f}%",
        f"{benchmark_metrics['best']:.2f}%",
        f"{benchmark_metrics['worst']:.2f}%"
    ]
})

print("\n📊 Portfolio vs S&P 500 Comparison\n")
display(comparison)

In [ ]:
portfolio_returns.head()

In [ ]:
# Ensure it’s a proper pandas Series with datetime index
portfolio_returns.index = pd.to_datetime(portfolio_returns.index)

# Example: fetch benchmark (S&P 500) returns automatically using quantstats
benchmark_returns = qs.utils.download_returns('SPY')


## 12. Advanced Analytics: Value at Risk & Stress Testing

In [ ]:
# Value at Risk (VaR) and Conditional VaR (CVaR) analysis
confidence_levels = [0.90, 0.95, 0.99]
current_portfolio_value = portfolio_df['Position_Value'].sum()

print("\n⚠️  Value at Risk (VaR) Analysis\n")
print("=" * 70)
print(f"Current Portfolio Value: ${current_portfolio_value:,.2f}\n")

var_data = []
for conf in confidence_levels:
    var = np.percentile(portfolio_returns, (1 - conf) * 100)
    cvar = portfolio_returns[portfolio_returns <= var].mean()
    
    var_dollar = var * current_portfolio_value
    cvar_dollar = cvar * current_portfolio_value
    
    print(f"{conf*100:.0f}% Confidence Level:")
    print(f"  VaR (1-day):  {var*100:.2f}% or ${abs(var_dollar):,.2f}")
    print(f"  CVaR (1-day): {cvar*100:.2f}% or ${abs(cvar_dollar):,.2f}")
    print()
    
    var_data.append({
        'Confidence': f"{conf*100:.0f}%",
        'VaR (%)': var * 100,
        'CVaR (%)': cvar * 100,
        'VaR ($)': abs(var_dollar),
        'CVaR ($)': abs(cvar_dollar)
    })

print("=" * 70)

# Visualize VaR
var_df = pd.DataFrame(var_data)
fig = go.Figure()

fig.add_trace(go.Bar(
    x=var_df['Confidence'],
    y=var_df['VaR (%)'],
    name='VaR',
    marker_color='orange'
))

fig.add_trace(go.Bar(
    x=var_df['Confidence'],
    y=var_df['CVaR (%)'],
    name='CVaR (Expected Shortfall)',
    marker_color='red'
))

fig.update_layout(
    title='Value at Risk at Different Confidence Levels',
    xaxis_title='Confidence Level',
    yaxis_title='1-Day Loss (%)',
    height=500,
    barmode='group'
)

fig.show()

## 13. Export Comprehensive Report

## 14. Summary & Key Insights

In [ ]:
# Generate executive summary
print("\n" + "="*80)
print(" " * 20 + "📊 EXECUTIVE PORTFOLIO SUMMARY 📊")
print("="*80 + "\n")

print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Analysis Period: {price_data.index[0].date()} to {price_data.index[-1].date()}")
print(f"Duration: {(price_data.index[-1] - price_data.index[0]).days} days\n")

print("💼 PORTFOLIO OVERVIEW")
print("-" * 80)
print(f"  Total Value:              ${portfolio_df['Position_Value'].sum():,.2f}")
print(f"  Number of Holdings:       {len(portfolio_df)}")
print(f"  Largest Position:         {portfolio_df.iloc[0]['Ticker']} ({portfolio_df.iloc[0]['Weight_Pct']:.1f}%)")
print(f"  Top 5 Holdings:           {portfolio_df.head(5)['Weight_Pct'].sum():.1f}% of portfolio\n")

print("📈 PERFORMANCE METRICS")
print("-" * 80)
print(f"  Total Return:             {(portfolio_cumulative_returns.iloc[-1] - 1)*100:>8.2f}%")
print(f"  CAGR:                     {qs.stats.cagr(portfolio_returns)*100:>8.2f}%")
print(f"  Sharpe Ratio:             {qs.stats.sharpe(portfolio_returns):>8.2f}")
print(f"  Sortino Ratio:            {qs.stats.sortino(portfolio_returns):>8.2f}")
print(f"  Max Drawdown:             {qs.stats.max_drawdown(portfolio_returns)*100:>8.2f}%")
print(f"  Volatility (Annual):      {qs.stats.volatility(portfolio_returns)*100:>8.2f}%")
print(f"  Win Rate:                 {qs.stats.win_rate(portfolio_returns)*100:>8.2f}%\n")

print("📊 VS S&P 500 BENCHMARK")
print("-" * 80)
portfolio_total = (portfolio_cumulative_returns.iloc[-1] - 1)*100
benchmark_total = (benchmark_cumulative.iloc[-1] - 1)*100
outperformance = portfolio_total - benchmark_total

print(f"  Portfolio Return:         {portfolio_total:>8.2f}%")
print(f"  S&P 500 Return:           {benchmark_total:>8.2f}%")
print(f"  Outperformance:           {outperformance:>8.2f}%")
print(f"  Beta:                     {beta:>8.2f}")
print(f"  Alpha:                    {alpha*100:>8.2f}%\n")

print("🎯 TOP PERFORMERS")
print("-" * 80)
top_performers = stock_metrics.nlargest(3, 'Total Return (%)')
for idx, (ticker, row) in enumerate(top_performers.iterrows(), 1):
    print(f"  {idx}. {ticker:8} {row['Total Return (%)']:>8.2f}%  (Sharpe: {row['Sharpe Ratio']:.2f})")

print("\n⚠️  UNDERPERFORMERS")
print("-" * 80)
bottom_performers = stock_metrics.nsmallest(3, 'Total Return (%)')
for idx, (ticker, row) in enumerate(bottom_performers.iterrows(), 1):
    print(f"  {idx}. {ticker:8} {row['Total Return (%)']:>8.2f}%  (Sharpe: {row['Sharpe Ratio']:.2f})")

print("\n💡 RISK ANALYSIS")
print("-" * 80)
print(f"  Average Correlation:      {avg_correlation:>8.4f}")
print(f"  Effective # of Stocks:    {effective_stocks:>8.2f}")
print(f"  VaR (95%, 1-day):         {np.percentile(portfolio_returns, 5)*100:>8.2f}%")
print(f"  CVaR (95%, 1-day):        {portfolio_returns[portfolio_returns <= np.percentile(portfolio_returns, 5)].mean()*100:>8.2f}%")

print("\n" + "="*80)
print(" " * 25 + "✅ ANALYSIS COMPLETE ✅")
print("="*80 + "\n")

## 🎉 Analysis Complete!

This comprehensive notebook has analyzed your portfolio using:
- **QuantStats**: Risk metrics, performance analytics, tearsheets
- **Plotly**: Interactive visualizations and dashboards  
- **Matplotlib/Seaborn**: Statistical plots and heatmaps
- **Pandas**: Data manipulation and analysis

### Next Steps:
1. Review the HTML tearsheet report for a comprehensive overview
2. Analyze the Excel workbook for detailed metrics
3. Use insights to rebalance or optimize your portfolio
4. Track performance over time by running this analysis periodically

### Customization Tips:
- Adjust the `start_date` to analyze different time periods
- Modify position sizes in the portfolio definition
- Compare against different benchmarks (QQQ, DIA, etc.)
- Add sector analysis by grouping stocks
- Implement Monte Carlo simulations for future projections